In [1]:
# 기본 패키지 불러오기 

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.express as px
import seaborn as sns 
plt.style.use("seaborn")
sns.set(font_scale=1)
sns.set_style("whitegrid")

# 그래프를 노트북 안에 그리기 위해 설정
%matplotlib inline

# 맥 OS 폰트 깨짐 방지 코드
from matplotlib import rc
import matplotlib as mpl 
import matplotlib.font_manager as fm 

rc('font', family='AppleGothic')
plt.rcParams['axes.unicode_minus'] = False

In [2]:
# # COLAB 폰트 깨짐 방지 코드
# !sudo apt-get install -y fonts-nanum
# !sudo fc-cache -fv
# !rm ~/.cache/matplotlib -rf
# # 설치후 런타임 재시작 필수 

# # matplotlib에 나눔고딕 폰트 적용
# import matplotlib.pyplot as plt
# import matplotlib.font_manager as fm
# # 기본 폰트 설정
# plt.rc('font', family='NanumBarunGothic')
# # 깨짐방지 테스트
# plt.text(0.5, 0.5, '안녕하세요, 한글 폰트 테스트입니다!', ha='center', va='center', size=15)
# plt.show()

# 데이터 준비

## 데이터 불러오기

In [3]:
# 데이터 불러오기
london_lists_raw = pd.read_csv('../../../../../data_weekly/inside_airbnb/London/listings.csv')
london_lists_summary_raw = pd.read_csv('../../../../../data_weekly/inside_airbnb/London/listings_outline.csv')
london_neighbourhoods_raw = pd.read_csv('../../../../../data_weekly/inside_airbnb/London/neighbourhoods.csv')
# london_reviews_raw = pd.read_csv('../../../../../data_weekly/inside_airbnb/London/reviews.csv')
# london_reviews_summary_raw = pd.read_csv('../../../../../data_weekly/inside_airbnb/London/reviews_outline.csv')

# 분석할 df 별도 복사
london_lists = london_lists_raw.copy()
# london_reviews = london_reviews_raw.copy()

# 출력
london_lists.head()

,id,listing_url,scrape_id,last_scraped,source,name,description,neighborhood_overview,picture_url,host_id,...,review_scores_communication,review_scores_location,review_scores_value,license,instant_bookable,calculated_host_listings_count,calculated_host_listings_count_entire_homes,calculated_host_listings_count_private_rooms,calculated_host_listings_count_shared_rooms,reviews_per_month
0,312761,https://www.airbnb.com/rooms/312761,20240319050633,2024-03-21,city scrape,Spacious Central London Apartment by Hoxton Sq...,"Very central location, in the middle of Shored...",Everything is so convenient and the area is al...,https://a0.muscache.com/pictures/miso/Hosting-...,1608226,...,5.00,4.89,4.93,NaN,f,1,1,0,0,0.86
1,13913,https://www.airbnb.com/rooms/13913,20240319050633,2024-03-20,city scrape,Holiday London DB Room Let-on going,My bright double bedroom with a large window h...,Finsbury Park is a friendly melting pot commun...,https://a0.muscache.com/pictures/miso/Hosting-...,54730,...,4.83,4.70,4.70,NaN,f,3,2,1,0,0.25
2,15400,https://www.airbnb.com/rooms/15400,20240319050633,2024-03-20,city scrape,Bright Chelsea Apartment. Chelsea!,Lots of windows and light. St Luke's Gardens ...,It is Chelsea.,https://a0.muscache.com/pictures/428392/462d26...,60302,...,4.83,4.93,4.74,NaN,f,1,1,0,0,0.54
3,159736,https://www.airbnb.com/rooms/159736,20240319050633,2024-03-20,city scrape,A double Room 5mins from King's College Hospital,Calm sunny double room with a queen size bed a...,We love that in Loughborough Junction we live ...,https://a0.muscache.com/pictures/1067303/d2300...,766056,...,4.89,4.34,4.66,NaN,f,4,0,4,0,0.62
4,165336,https://www.airbnb.com/rooms/165336,20240319050633,2024-03-21,city scrape,Charming Flat in Notting Hill,A stylish apartment close to Portobello market...,"Notting Hill has many cafes, bars and restaura...",https://a0.muscache.com/pictures/60757460/47f8...,761400,...,4.96,4.87,4.71,NaN,f,1,1,0,0,1.57


## 데이터 불러온 최종일자 확인 

In [4]:
london_lists_raw['last_scraped'].unique()

array(['2024-03-21', '2024-03-20', '2024-03-22', '2024-03-19'],
      dtype=object)

슈퍼호스트 선정 기준 날짜 년 4회(분기별)
- 1월 1일 / 4월 1일 / 7월 1일 / 10월 1일 

## 분석에서 사용하지 않을 컬럼 삭제
- 추가한 컬럼 : description, host_has_profile_pic, host_identity_verified

In [5]:
# 1차 선별한 컬럼 25개 중 22개 2차 선별 + 1개(숙소_예약가능_여부)
# 최근 리뷰가 작성되었는지를 보기 위해 2개 컬럼 추가_240529 >> 'number_of_reviews_ltm' // 'last_review'
# 'listing_url' 컬럼 london_lists에 적용_240530
# 240602_추가한 컬럼 : description, host_has_profile_pic, host_identity_verified
# 위도, 경도 추가 240608


columns_selected = ['id', 'listing_url', 'description', 'host_id', 'host_response_time', 'host_response_rate', 'host_is_superhost', 'host_has_profile_pic', 'host_identity_verified', 'neighbourhood_cleansed', 'latitude', 'longitude', 'property_type', \
    'room_type', 'accommodates', 'bedrooms', 'beds', 'amenities', 'price', 'has_availability', 'number_of_reviews', 'number_of_reviews_ltm', 'number_of_reviews_l30d', 'last_review', 'review_scores_rating', 'review_scores_accuracy', \
    'review_scores_cleanliness', 'review_scores_checkin', 'review_scores_communication', 'review_scores_location', 'review_scores_value', 'reviews_per_month']

# 'host_acceptance_rate', 'minimum_nights', 'maximum_nights', 'has_availability', 'host_total_listings_count', 'host_has_profile_pic', 'bathrooms',

# 선별한 컬럼만 적용
london_lists = london_lists[columns_selected]


# 변수 정리 
london_lists = london_lists.rename(columns={
    'id': '숙소_id',
    'listing_url' : '숙소_url',
    'description' : '숙소_설명',
    'host_id': '호스트_id',
    'host_response_time': '답변_평균시간',
    'host_response_rate': '문의_응답률',
    # 'host_acceptance_rate': '예약_수락률',
    'host_is_superhost': '슈퍼호스트',
    # 'host_total_listings_count': '숙소_수',
    'host_has_profile_pic' : '프로필_사진',
    'host_identity_verified' : '호스트_신원',
    'neighbourhood_cleansed': '숙소_지역',
    'latitude' : '위도',
    'longitude' : '경도',
    'property_type' : '숙소_특징',
    'room_type': '숙소_유형',
    'accommodates': '수용_인원수',
    # 'bathrooms': '욕실수',
    'bedrooms': '침실수',
    'beds': '침대수',
    'amenities': '편의시설',
    'price': '숙소_가격',
    # 'minimum_nights': '최소_숙박일',
    # 'maximum_nights': '최대_숙박일',
    'has_availability' : '예약_가능여부',
    'number_of_reviews': '리뷰수',
    'number_of_reviews_ltm' : '12개월_리뷰수',
    'number_of_reviews_l30d': '30일_리뷰수',
    'last_review' : '마지막_리뷰',
    'review_scores_rating': '리뷰점수',
    'review_scores_accuracy': '숙소_정확성_리뷰점수',
    'review_scores_cleanliness': '숙소_청결도_리뷰점수',
    'review_scores_checkin': '숙소_체크인_리뷰점수',
    'review_scores_communication': '숙소_소통_리뷰점수',
    'review_scores_location': '숙소_위치_리뷰점수',
    'review_scores_value': '숙소_가격_리뷰점수',
    'reviews_per_month': '평균_리뷰수'
 
})

# 컬럼명 확인
london_lists.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 90852 entries, 0 to 90851
Data columns (total 32 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   숙소_id        90852 non-null  int64  
 1   숙소_url       90852 non-null  object 
 2   숙소_설명        87851 non-null  object 
 3   호스트_id       90852 non-null  int64  
 4   답변_평균시간      57797 non-null  object 
 5   문의_응답률       57797 non-null  object 
 6   슈퍼호스트        90463 non-null  object 
 7   프로필_사진       90847 non-null  object 
 8   호스트_신원       90847 non-null  object 
 9   숙소_지역        90852 non-null  object 
 10  위도           90852 non-null  float64
 11  경도           90852 non-null  float64
 12  숙소_특징        90852 non-null  object 
 13  숙소_유형        90852 non-null  object 
 14  수용_인원수       90852 non-null  int64  
 15  침실수          78118 non-null  float64
 16  침대수          57837 non-null  float64
 17  편의시설         90852 non-null  object 
 18  숙소_가격        57885 non-null  object 
 19  예약_가

## 데이터 전처리

### 결측치 처리
- 숙소 유형  >> 모든 데이터 사용
- (이외 상세 내용은 가격_관계분석_런던2.ipynb확인요망)

In [6]:
print(f'런던 데이터 전체 개수: {london_lists.shape[0]}개')
print("-" * 60)

# 리뷰수가 0인 데이터 제거
condition_review_0 = london_lists['리뷰수'] == 0
london_lists = london_lists[~condition_review_0]
print(f'리뷰가 0인 데이터 제거 후 데이터 개수: {london_lists.shape[0]}개')
print("-" * 60)

# 카테고리 타입 데이터 비율 및 개수 세는 함수 선언
def CategoricalData_Count_Ratio(name_of_dataframe, name_of_column):

    print(f'전체 숙소 개수: {name_of_dataframe.shape[0]}개')
    print()

    type_list = name_of_dataframe[name_of_column].unique().tolist()

    for type in type_list:
        condition_type = name_of_dataframe[name_of_column] == type
        ratio_type = round((name_of_dataframe[condition_type].shape[0] / name_of_dataframe.shape[0]) * 100, 2)
        print(f'{name_of_column}이 {type}인 숙소의 개수: {name_of_dataframe[condition_type].shape[0]}개')
        print(f'{name_of_column}이 {type}인 숙소의 비율 : {ratio_type}%')
        print()


# # 숙소 유형 [Entire home/apt & Private room] 데이터만 선별
# condition_room_entirehomeapt = london_lists['숙소_유형'] == 'Entire home/apt'
# condition_room_privateroom = london_lists['숙소_유형'] == 'Private room'
# london_lists = london_lists[(condition_room_entirehomeapt | condition_room_privateroom)]
# print('[Entire home/apt / Private room] 데이터만 선별 : ')
# CategoricalData_Count_Ratio(london_lists, '숙소_유형')
# print("-" * 60)

# 예약_가능여부 Null값 제거 및 예약 불가능 숙소 제거
london_lists = london_lists[london_lists["예약_가능여부"].notnull()]
condition_availability_f = london_lists['예약_가능여부'] == 'f'
london_lists = london_lists[~condition_availability_f]
print(f'Null값 제거 및 예약 불가능 숙소 제거 후 데이터 개수: {london_lists.shape[0]}개')
print("-" * 60)

# 숙소 가격 타입 변환 >> float
london_lists['숙소_가격'] = london_lists['숙소_가격'].str.lstrip('$').str.replace(',', '').astype('float')

# 가격 null값 제거 
condition_price_notnull = london_lists['숙소_가격'].notnull()
london_lists = london_lists[condition_price_notnull] 
print(f'가격 Null값 제거 후 데이터 개수: {london_lists.shape[0]}개')
print("-" * 60)

# 슈퍼호스트 결측치 제거
london_lists = london_lists.dropna(subset=['슈퍼호스트'])
print(f'슈퍼호스트 Null값 제거 후 데이터 개수: {london_lists.shape[0]}개')
print("-" * 60)


# 침대수, 침실수 결측치 제거
london_lists = london_lists.dropna(subset=['침실수', '침대수'])
print(f'침실수, 침대수 Null값 제거 후 데이터 개수: {london_lists.shape[0]}개')
print("-" * 60)


# 각 리뷰 점수 컬럼 Null 값 제거 
type_review_null_any = london_lists[['숙소_정확성_리뷰점수', '숙소_청결도_리뷰점수', '숙소_체크인_리뷰점수', '숙소_소통_리뷰점수', '숙소_위치_리뷰점수', '숙소_가격_리뷰점수']].isnull().any(axis=1)
london_lists = london_lists[~type_review_null_any]
print(f'각 리뷰점수 Null값 제거 후 데이터 개수: {london_lists.shape[0]}개')
print("-" * 60)

런던 데이터 전체 개수: 90852개
------------------------------------------------------------
리뷰가 0인 데이터 제거 후 데이터 개수: 65869개
------------------------------------------------------------
Null값 제거 및 예약 불가능 숙소 제거 후 데이터 개수: 64463개
------------------------------------------------------------
가격 Null값 제거 후 데이터 개수: 43720개
------------------------------------------------------------
슈퍼호스트 Null값 제거 후 데이터 개수: 43454개
------------------------------------------------------------
침실수, 침대수 Null값 제거 후 데이터 개수: 43367개
------------------------------------------------------------
각 리뷰점수 Null값 제거 후 데이터 개수: 43345개
------------------------------------------------------------


### 이상치 처리
- 기존 전처리에서 각 숙소_유형 별 상위 10개(개수) 선별하지 않음 >> 모든 데이터 사용
- 욕실수, 침실수, 침대수 이상치 처리도 다 해버리자~ (스케일러 사용 관련)
- (이외 이상치 처리 상세 내용은 가격_관계분석_런던2.ipynb확인요망)

In [7]:
# # 각 숙소_유형 별 상위 10개(개수) 숙소_특징 데이터만 선별 
# # 조건 생성
# sorted_PrivateRoom = london_lists.groupby(['숙소_유형', '숙소_특징']).size().reset_index(name='count').sort_values(by=['숙소_유형', 'count'], ascending=False, ignore_index=True)[:28].reset_index(drop=True)
# sorted_EntireHomeApt = london_lists.groupby(['숙소_유형', '숙소_특징']).size().reset_index(name='count').sort_values(by=['숙소_유형', 'count'], ascending=False, ignore_index=True)[28:].reset_index(drop=True)
# sorted_PrivateRoom_top10 = sorted_PrivateRoom['숙소_특징'][:10].values
# sorted_EntireHomeApt_top10 = sorted_EntireHomeApt['숙소_특징'][:10].values
# condition_PrivateRoom_top10 = london_lists['숙소_특징'].isin(sorted_PrivateRoom_top10)
# condition_EntireHomeApt_top10 = london_lists['숙소_특징'].isin(sorted_EntireHomeApt_top10)
# # 데이터 제거
# london_lists = london_lists[(condition_PrivateRoom_top10 | condition_EntireHomeApt_top10)]
# print(f'숙소_유형별 숙소_특징 상위 10개씩 20개 선별 후 데이터 개수: {london_lists.shape[0]}개')
# print("-" * 60)


# 12개월_리뷰수가 0인 경우 제거 
condition_reviewforyear_0 = london_lists['12개월_리뷰수'] == 0
london_lists = london_lists[~condition_reviewforyear_0]
print(f'12개월_리뷰수가 0인 데이터 제거 후 데이터 개수: {london_lists.shape[0]}개')
print("-" * 60)


# 수용인원수 이상치 처리 (숙소 유형별 처리)
accommodates_type_list = london_lists['숙소_유형'].unique().tolist()

def delete_outlier_IQR_InnerFence(column_name):
    # 전역 변수 선언
    global london_lists
    # 숙소 유형별 Inner Fencs Out Lier 제거 
    for type in accommodates_type_list:
        condition_room_type = london_lists['숙소_유형'] == type
        Q1 = london_lists[column_name][condition_room_type].quantile(.25)
        Q3 = london_lists[column_name][condition_room_type].quantile(.75)
        IQR = Q3 - Q1
        condition_accommodates_upper_outliers = london_lists[column_name][condition_room_type] > (Q3 + IQR * 1.5)
        accommodates_upper_outliers = london_lists[condition_room_type][condition_accommodates_upper_outliers]
        upper_outlier_indices = accommodates_upper_outliers.index
        london_lists = london_lists.drop(index=upper_outlier_indices)
        print(f'{type} 데이터 중 {column_name} Upper_outlier 제거 후 데이터 개수 : {london_lists.shape[0]}개')

delete_outlier_IQR_InnerFence('수용_인원수')
print("-" * 60)

# 침실수 이상치 처리
delete_outlier_IQR_InnerFence('침실수')
print("-" * 60)

# 침대수 이상치 처리 
delete_outlier_IQR_InnerFence('침대수')
print("-" * 60)


# 가격 이상치 처리 (숙소 유형별 처리)

accommodates_type_list = london_lists['숙소_유형'].unique().tolist()

def delete_outlier_IQR_OuterFence(column_name):
    # 전역 변수 선언
    global london_lists
    # 숙소 유형별 Outer Fencs Out Lier 제거 
    for type in accommodates_type_list:
        condition_room_type = london_lists['숙소_유형'] == type
        Q1 = london_lists[column_name][condition_room_type].quantile(.25)
        Q3 = london_lists[column_name][condition_room_type].quantile(.75)
        IQR = Q3 - Q1
        condition_accommodates_upper_outliers = london_lists[column_name][condition_room_type] > (Q3 + IQR * 3)
        accommodates_upper_outliers = london_lists[condition_room_type][condition_accommodates_upper_outliers]
        upper_outlier_indices = accommodates_upper_outliers.index
        london_lists = london_lists.drop(index=upper_outlier_indices)
        print(f'{type} 데이터 중 {column_name} Upper_outlier 제거 후 데이터 개수 : {london_lists.shape[0]}개')

delete_outlier_IQR_OuterFence('숙소_가격')
print("-" * 60)

# 가격이 0원인 숙소 제거 
condition_price_0 = london_lists['숙소_가격'] == 0
london_lists = london_lists[~condition_price_0]
print(f'"숙소_가격"이 0원인 데이터 제거 후 데이터 개수 : {london_lists.shape[0]}개')

12개월_리뷰수가 0인 데이터 제거 후 데이터 개수: 38982개
------------------------------------------------------------
Entire home/apt 데이터 중 수용_인원수 Upper_outlier 제거 후 데이터 개수 : 38426개
Private room 데이터 중 수용_인원수 Upper_outlier 제거 후 데이터 개수 : 37698개
Hotel room 데이터 중 수용_인원수 Upper_outlier 제거 후 데이터 개수 : 37695개
Shared room 데이터 중 수용_인원수 Upper_outlier 제거 후 데이터 개수 : 37680개
------------------------------------------------------------
Entire home/apt 데이터 중 침실수 Upper_outlier 제거 후 데이터 개수 : 36640개
Private room 데이터 중 침실수 Upper_outlier 제거 후 데이터 개수 : 36326개
Hotel room 데이터 중 침실수 Upper_outlier 제거 후 데이터 개수 : 36317개
Shared room 데이터 중 침실수 Upper_outlier 제거 후 데이터 개수 : 36316개
------------------------------------------------------------
Entire home/apt 데이터 중 침대수 Upper_outlier 제거 후 데이터 개수 : 36245개
Private room 데이터 중 침대수 Upper_outlier 제거 후 데이터 개수 : 35325개
Hotel room 데이터 중 침대수 Upper_outlier 제거 후 데이터 개수 : 35324개
Shared room 데이터 중 침대수 Upper_outlier 제거 후 데이터 개수 : 35320개
------------------------------------------------------------
Entire home

### 컬럼 데이터 타입 정리

In [8]:
# 호스트_신원 / 프로필_사진 / 답변_평균시간 / 문의_응답률 컬럼 다시 확인 
new_column_list = ['호스트_신원', '프로필_사진', '답변_평균시간', '문의_응답률']
for new_column in new_column_list:
    print(f'{new_column}의 UNIQUE값:')
    print(london_lists[new_column].unique())
    print(f'{new_column}의 UNIQUE값의 개수: {london_lists[new_column].nunique()}')
    print('-' * 50)

호스트_신원의 UNIQUE값:
['t' 'f']
호스트_신원의 UNIQUE값의 개수: 2
--------------------------------------------------
프로필_사진의 UNIQUE값:
['t' 'f']
프로필_사진의 UNIQUE값의 개수: 2
--------------------------------------------------
답변_평균시간의 UNIQUE값:
['within a few hours' 'within a day' 'within an hour' 'a few days or more'
 nan]
답변_평균시간의 UNIQUE값의 개수: 4
--------------------------------------------------
문의_응답률의 UNIQUE값:
['100%' '86%' '0%' '96%' nan '85%' '91%' '90%' '97%' '80%' '95%' '93%'
 '84%' '71%' '67%' '62%' '92%' '75%' '13%' '7%' '33%' '89%' '70%' '60%'
 '50%' '63%' '77%' '25%' '40%' '98%' '88%' '57%' '10%' '99%' '94%' '48%'
 '83%' '56%' '52%' '20%' '29%' '87%' '11%' '81%' '65%' '27%' '30%' '73%'
 '58%' '14%' '46%' '79%' '64%' '38%' '22%' '43%' '76%' '82%' '78%' '72%'
 '26%' '59%' '17%' '35%' '47%' '44%' '74%' '68%' '3%' '53%' '42%' '69%'
 '5%' '19%' '8%' '9%' '21%' '61%']
문의_응답률의 UNIQUE값의 개수: 77
--------------------------------------------------


In [9]:
# 컬럼 데이터 타입 변환 

# ID => 문자로 변환
london_lists['숙소_id'] = london_lists['숙소_id'].astype('str')
london_lists['호스트_id'] = london_lists['호스트_id'].astype('str')

# 숙소_지역, 숙소_특징, 숙소_유형 >> 범주형 데이터로 변환 
for column in ['슈퍼호스트', '숙소_지역', '숙소_특징', '숙소_유형', '호스트_신원', '프로필_사진', '답변_평균시간']:
    london_lists[column] = london_lists[column].astype('category')

In [10]:
london_lists.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 34639 entries, 0 to 90735
Data columns (total 32 columns):
 #   Column       Non-Null Count  Dtype   
---  ------       --------------  -----   
 0   숙소_id        34639 non-null  object  
 1   숙소_url       34639 non-null  object  
 2   숙소_설명        34042 non-null  object  
 3   호스트_id       34639 non-null  object  
 4   답변_평균시간      32154 non-null  category
 5   문의_응답률       32154 non-null  object  
 6   슈퍼호스트        34639 non-null  category
 7   프로필_사진       34639 non-null  category
 8   호스트_신원       34639 non-null  category
 9   숙소_지역        34639 non-null  category
 10  위도           34639 non-null  float64 
 11  경도           34639 non-null  float64 
 12  숙소_특징        34639 non-null  category
 13  숙소_유형        34639 non-null  category
 14  수용_인원수       34639 non-null  int64   
 15  침실수          34639 non-null  float64 
 16  침대수          34639 non-null  float64 
 17  편의시설         34639 non-null  object  
 18  숙소_가격        34639 non-nul

### 추가 컬럼 결측치 처리 및 전처리

In [11]:
# 숙소_설명 null값 제거 
condition_description_notnull = london_lists['숙소_설명'].notnull()
london_lists = london_lists[condition_description_notnull] 
print(f'숙소_설명 Null값 제거 후 데이터 개수: {london_lists.shape[0]}개')
print("-" * 60)

# 답변_평균시간 null값 제거 & 문의_응답률 Null값과 답변_평균시간 null값 동일
condition_responsetime_notnull = london_lists['답변_평균시간'].notnull()
london_lists = london_lists[condition_responsetime_notnull] 
print(f'답변_평균시간 & 문의_응답률 Null값 제거 후 데이터 개수: {london_lists.shape[0]}개')
print("-" * 60)

# 문의_응답률 타입 변환 >> float
london_lists['문의_응답률'] = london_lists['문의_응답률'].str.rstrip('%').astype('float')

숙소_설명 Null값 제거 후 데이터 개수: 34042개
------------------------------------------------------------
답변_평균시간 & 문의_응답률 Null값 제거 후 데이터 개수: 31599개
------------------------------------------------------------


In [12]:
# 문의_응답률이 0인 경우
condition_responserate_0 = london_lists['문의_응답률'] == 0
print(f"문의_응답률이 0%인 숙소의 개수: {london_lists[condition_responserate_0].shape[0]}개")
london_lists = london_lists[~condition_responserate_0]
print(f"문의_응답률이 0%인 숙소 제거 후 데이터 개수: {london_lists.shape[0]}개")

문의_응답률이 0%인 숙소의 개수: 331개
문의_응답률이 0%인 숙소 제거 후 데이터 개수: 31268개


In [13]:
london_lists.isnull().sum()

숙소_id          0
숙소_url         0
숙소_설명          0
호스트_id         0
답변_평균시간        0
문의_응답률         0
슈퍼호스트          0
프로필_사진         0
호스트_신원         0
숙소_지역          0
위도             0
경도             0
숙소_특징          0
숙소_유형          0
수용_인원수         0
침실수            0
침대수            0
편의시설           0
숙소_가격          0
예약_가능여부        0
리뷰수            0
12개월_리뷰수       0
30일_리뷰수        0
마지막_리뷰         0
리뷰점수           0
숙소_정확성_리뷰점수    0
숙소_청결도_리뷰점수    0
숙소_체크인_리뷰점수    0
숙소_소통_리뷰점수     0
숙소_위치_리뷰점수     0
숙소_가격_리뷰점수     0
평균_리뷰수         0
dtype: int64

### 파생변수 생성 및 전처리 

#### 편의시설_개수

In [14]:
# 편의시설_개수 파생변수 생성
london_lists['편의시설_개수'] = london_lists['편의시설'].apply(lambda x: len(eval(x)))

# 편의시설 데이터값 리스트화
london_lists['편의시설'] = london_lists['편의시설'].apply(lambda x: eval(x))

In [15]:
# 이상치 처리 
delete_outlier_IQR_InnerFence('편의시설_개수')
print("-" * 60)

Entire home/apt 데이터 중 편의시설_개수 Upper_outlier 제거 후 데이터 개수 : 31209개
Private room 데이터 중 편의시설_개수 Upper_outlier 제거 후 데이터 개수 : 31199개
Hotel room 데이터 중 편의시설_개수 Upper_outlier 제거 후 데이터 개수 : 31198개
Shared room 데이터 중 편의시설_개수 Upper_outlier 제거 후 데이터 개수 : 31195개
------------------------------------------------------------


#### 숙소_설명_글자수

In [16]:
from bs4 import BeautifulSoup
import re

def remove_html_tags(text):
    soup = BeautifulSoup(text, "html.parser")
    return soup.get_text()

def remove_non_alphanumeric(text):
    return re.sub(r'[^A-Za-z0-9 ]+', '', text)

# HTML 태그 제거
london_lists['숙소_설명'] = london_lists['숙소_설명'].apply(remove_html_tags)

# # 텍스트 외의 형식 제거
# london_lists['cleaned_description'] = london_lists['cleaned_description'].apply(remove_non_alphanumeric)

# 글자 수 세기
london_lists['숙소_설명_글자수'] = london_lists['숙소_설명'].apply(len)

# 출력
london_lists[['숙소_설명', '숙소_설명_글자수']].head()

/Users/minseokpark/opt/anaconda3/lib/python3.9/site-packages/bs4/__init__.py:435: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  warnings.warn(


,숙소_설명,숙소_설명_글자수
0,"Very central location, in the middle of Shored...",436
1,My bright double bedroom with a large window h...,299
2,Lots of windows and light. St Luke's Gardens ...,254
3,Calm sunny double room with a queen size bed a...,399
4,A stylish apartment close to Portobello market...,113


In [17]:
# 이상치 처리 

delete_outlier_IQR_InnerFence('숙소_설명_글자수')
print("-" * 60)

Entire home/apt 데이터 중 숙소_설명_글자수 Upper_outlier 제거 후 데이터 개수 : 31116개
Private room 데이터 중 숙소_설명_글자수 Upper_outlier 제거 후 데이터 개수 : 31101개
Hotel room 데이터 중 숙소_설명_글자수 Upper_outlier 제거 후 데이터 개수 : 31101개
Shared room 데이터 중 숙소_설명_글자수 Upper_outlier 제거 후 데이터 개수 : 31101개
------------------------------------------------------------


## 분석 기준(파생변수) 생성
- 숙소_등급

In [18]:
# 인덱스 정리 
london_lists.reset_index(drop=True, inplace=True)
london_lists[:10]

,숙소_id,숙소_url,숙소_설명,호스트_id,답변_평균시간,문의_응답률,슈퍼호스트,프로필_사진,호스트_신원,숙소_지역,...,리뷰점수,숙소_정확성_리뷰점수,숙소_청결도_리뷰점수,숙소_체크인_리뷰점수,숙소_소통_리뷰점수,숙소_위치_리뷰점수,숙소_가격_리뷰점수,평균_리뷰수,편의시설_개수,숙소_설명_글자수
0,312761,https://www.airbnb.com/rooms/312761,"Very central location, in the middle of Shored...",1608226,within a few hours,100.0,t,t,t,Hackney,...,4.96,4.97,4.93,4.96,5.00,4.89,4.93,0.86,46,436
1,13913,https://www.airbnb.com/rooms/13913,My bright double bedroom with a large window h...,54730,within a few hours,86.0,t,t,t,Islington,...,4.80,4.73,4.73,4.75,4.83,4.70,4.70,0.25,55,299
2,15400,https://www.airbnb.com/rooms/15400,Lots of windows and light. St Luke's Gardens ...,60302,within a day,100.0,f,t,t,Kensington and Chelsea,...,4.80,4.85,4.88,4.88,4.83,4.93,4.74,0.54,25,254
3,159736,https://www.airbnb.com/rooms/159736,Calm sunny double room with a queen size bed a...,766056,within an hour,100.0,t,t,t,Lambeth,...,4.72,4.79,4.66,4.75,4.89,4.34,4.66,0.62,26,399
4,165336,https://www.airbnb.com/rooms/165336,A stylish apartment close to Portobello market...,761400,within an hour,100.0,t,t,t,Kensington and Chelsea,...,4.74,4.88,4.79,4.94,4.96,4.87,4.71,1.57,40,113
5,17402,https://www.airbnb.com/rooms/17402,"You'll have a great time in this beautiful, cl...",67564,within an hour,100.0,t,t,t,Westminster,...,4.77,4.83,4.72,4.72,4.72,4.89,4.61,0.35,38,378
6,34935,https://www.airbnb.com/rooms/34935,Whole private apartment (not shared) in a bui...,133271,within an hour,96.0,f,t,t,Tower Hamlets,...,4.55,4.70,4.67,4.77,4.75,4.58,4.54,0.73,43,466
7,36299,https://www.airbnb.com/rooms/36299,3 Bed House with garden close to Thames river ...,155938,within a day,100.0,t,t,t,Richmond upon Thames,...,4.80,4.77,4.78,4.94,4.96,4.90,4.62,0.65,34,255
8,173082,https://www.airbnb.com/rooms/173082,A lovely 1 bedroom flat above an art gallery i...,813141,within an hour,100.0,f,t,t,Hackney,...,4.75,4.86,4.76,4.95,4.90,4.71,4.71,1.42,44,336
9,174132,https://www.airbnb.com/rooms/174132,inside flat is one staircase to go downstairs ...,831866,within an hour,100.0,f,t,t,Westminster,...,4.65,4.78,4.75,4.73,4.74,4.88,4.64,1.84,32,109


### 리뷰점수 상하위 10% 조회

In [19]:
import plotly.express as px
import plotly.graph_objs as go


# '리뷰점수' 데이터 분포를 시각화
fig = px.histogram(london_lists, x='리뷰점수', nbins=50, title='리뷰점수 분포')
fig.update_layout(bargap=0.2, 
                  width=800, # 그래프 크기 조정
                  height=500,
                  xaxis_title='리뷰점수',
                  yaxis_title='빈도')
fig.show()

In [20]:
# 상위 10%와 하위 10%를 나누기 위한 경계값 계산
lower_quartile = london_lists['리뷰점수'].quantile(0.10)
upper_quartile = london_lists['리뷰점수'].quantile(0.90)

# 파생변수 추가: '리뷰점수_등급'
def review_grade(score):
    if score <= lower_quartile:
        return 'Low'
    elif score >= upper_quartile:
        return 'Top'
    else:
        return 'Middle'

london_lists['리뷰점수_등급'] = london_lists['리뷰점수'].apply(review_grade)

# 결과 확인
print(london_lists[['리뷰점수', '리뷰점수_등급']].head())


   리뷰점수 리뷰점수_등급
0  4.96  Middle
1  4.80  Middle
2  4.80  Middle
3  4.72  Middle
4  4.74  Middle


In [21]:
# 리뷰점수_등급 카테고리 변환 
london_lists['리뷰점수_등급'] = london_lists['리뷰점수_등급'].astype('category')

# 리뷰점수_등급 별 '리뷰점수' 기초 통계 확인
리뷰등급별_리뷰점수 = \
london_lists.groupby('리뷰점수_등급')['리뷰점수'].agg(
    count='size',
    mean='mean',
    std='std',
    min='min',
    max='max',
    median='median'
).reset_index().sort_values(by= 'count', ascending=False).reset_index(drop=True)

리뷰등급별_리뷰점수

,리뷰점수_등급,count,mean,std,min,max,median
0,Middle,19262,4.722528,0.181343,4.26,4.99,4.76
1,Top,8657,5.000000,0.000000,5.00,5.00,5.00
2,Low,3182,3.716534,0.683635,1.00,4.25,4.00


In [22]:
# 리뷰점수_등급 별 '리뷰수' 기초 통계 확인
리뷰등급별_리뷰수 = \
london_lists.groupby('리뷰점수_등급')['리뷰수'].agg(
    count='size',
    mean='mean',
    std='std',
    min='min',
    max='max',
    median='median'
).reset_index().sort_values(by= 'count', ascending=False).reset_index(drop=True)

리뷰등급별_리뷰수

,리뷰점수_등급,count,mean,std,min,max,median
0,Middle,19262,45.389160,65.325933,2,1361,23.0
1,Top,8657,6.348273,11.060010,1,397,3.0
2,Low,3182,10.661534,26.452834,1,802,4.0


1. 리뷰점수 만으로는 에어비앤비에서 이야기하는 상위 10% 숙소를 추려낼 수 없음 
2. 게스트 선호 숙소 기준을 가져와서 좀 더 추려야할 듯 

### 게스트_선호

In [23]:
# 최근 2년간 리뷰가 5개 이상인 조건 설정 
# 1) 마지막 리뷰가 '2022-04-01' 이후인 데이터 조건
london_lists['마지막_리뷰'] = pd.to_datetime(london_lists['마지막_리뷰'])
start_date = '2022-04-01'
condition_lastreview_guestfavorite = london_lists['마지막_리뷰'] >= start_date

# 2) 12개월 간 리뷰수가 0이 아닌 데이터 조건
condition_reviewforyear = london_lists['12개월_리뷰수'] != 0

# 3) 전체 리뷰수가 5 이상인 데이터 조건
condition_review_5 = london_lists['리뷰수'] >= 5

# 리뷰 점수가 4.9 이상인 데이터 조건
condition_score_guestfavorite = london_lists['리뷰점수'] >= 4.9

# 슈퍼호스트인 데이터
condition_superhost = london_lists['슈퍼호스트'] == 't'

print(f'슈퍼호스트 조건까지 집어넣은 게스트_선호 데이터 개수 :{london_lists[condition_lastreview_guestfavorite & condition_reviewforyear & condition_review_5 & condition_score_guestfavorite & condition_superhost].shape[0]}개')
print(f'슈퍼호스트 조건은 제외한 게스트_선호 데이터 개수 :{london_lists[condition_lastreview_guestfavorite & condition_reviewforyear & condition_review_5 & condition_score_guestfavorite].shape[0]}개')

슈퍼호스트 조건까지 집어넣은 게스트_선호 데이터 개수 :3901개
슈퍼호스트 조건은 제외한 게스트_선호 데이터 개수 :6891개


In [24]:
# 파생변수 생성
london_lists['게스트_선호'] = 'f'

# 슈퍼호스트 조건 포함 
london_lists.loc[
    condition_lastreview_guestfavorite &
    condition_reviewforyear &
    condition_review_5 &
    condition_score_guestfavorite &
    condition_superhost, '게스트_선호'] = 't'

# # 슈퍼호스트 조건을 포함하지 않는 게스트 선호 숙소
# london_lists.loc[
#     condition_lastreview_guestfavorite &
#     condition_reviewforyear &
#     condition_review_5 &
#     condition_score_guestfavorite &
#     ~condition_superhost, '게스트_선호'] = 'm'

# 기초 통계량 조회 

# 게스트_선호 카테고리 변환 
london_lists['게스트_선호'] = london_lists['게스트_선호'].astype('category')

# 게스트_선호 별 '리뷰점수' 기초 통계 확인
게스트선호별_리뷰점수 = \
london_lists.groupby('게스트_선호')['리뷰점수'].agg(
    count='size',
    mean='mean',
    std='std',
    min='min',
    max='max',
    median='median'
).reset_index().sort_values(by= 'count', ascending=False).reset_index(drop=True)

게스트선호별_리뷰점수


,게스트_선호,count,mean,std,min,max,median
0,f,27200,4.659020,0.456620,1.0,5.0,4.78
1,t,3901,4.960523,0.036308,4.9,5.0,4.96


In [25]:

# 게스트_선호 별 '리뷰점수' 기초 통계 확인
게스트선호별_리뷰수 = \
london_lists.groupby('게스트_선호')['리뷰수'].agg(
    count='size',
    mean='mean',
    std='std',
    min='min',
    max='max',
    median='median'
).reset_index().sort_values(by= 'count', ascending=False).reset_index(drop=True)

게스트선호별_리뷰수

,게스트_선호,count,mean,std,min,max,median
0,f,27200,28.845221,54.753262,1,1361,10.0
1,t,3901,45.777493,58.894453,5,1011,25.0


In [26]:
# 게스트 선호 숙소 중 리뷰가 5.0인 숙소 개수 확인

# 조건 설정
condition_guestfavorite_t = london_lists['게스트_선호'] == 't'
condition_score_5 = london_lists['리뷰점수'] == 5.0

# 개수 출력
print(f'게스트 선호 숙소 중 리뷰가 5.0인 숙소 개수 : {london_lists[condition_guestfavorite_t & condition_score_5].shape[0]}개')

게스트 선호 숙소 중 리뷰가 5.0인 숙소 개수 : 1452개


### 숙소_등급

- 상위 10% 기준 : 게스트 선호 숙소 T 
- 하위 10% 기준 : 리뷰 등급 하위 10% 
- 우선 비교를 통해 유의미한 값이 나오는지 확인하는 것을 목표로 함 

In [27]:
# 하위 10%를 나누기 위한 경계값 계산
lower_quartile = london_lists['리뷰점수'].quantile(0.10)

# 파생변수 추가: '숙소_등급'
def review_grade(score):
    if score <= lower_quartile:
        return 'Low'
    else:
        return 'Middle'

london_lists['숙소_등급'] = london_lists['리뷰점수'].apply(review_grade)

# 상위 10% 값 추가 : 게스트 선호 숙소 == t
london_lists.loc[
    condition_lastreview_guestfavorite &
    condition_reviewforyear &
    condition_review_5 &
    condition_score_guestfavorite &
    condition_superhost, '숙소_등급'] = 'Top'

# 컬럼 제거 
london_lists.drop(columns='리뷰점수_등급', inplace=True)

# 결과 확인
숙소등급별_리뷰점수 = \
london_lists.groupby('숙소_등급')['리뷰점수'].agg(
    count='size',
    mean='mean',
    std='std',
    min='min',
    max='max',
    median='median'
).reset_index().sort_values(by= 'count', ascending=False).reset_index(drop=True)

숙소등급별_리뷰점수

,숙소_등급,count,mean,std,min,max,median
0,Middle,24018,4.783885,0.202351,4.26,5.00,4.82
1,Top,3901,4.960523,0.036308,4.90,5.00,4.96
2,Low,3182,3.716534,0.683635,1.00,4.25,4.00


In [28]:
# london_lists.to_csv('../Data/london_lists_preprocessed.csv')

In [29]:
# Top과 Low의 비교를 위한 별도의 DF 생성

london_toplow = london_lists[london_lists['숙소_등급'].isin(['Top', 'Low'])]
london_toplow[:10]

,숙소_id,숙소_url,숙소_설명,호스트_id,답변_평균시간,문의_응답률,슈퍼호스트,프로필_사진,호스트_신원,숙소_지역,...,숙소_청결도_리뷰점수,숙소_체크인_리뷰점수,숙소_소통_리뷰점수,숙소_위치_리뷰점수,숙소_가격_리뷰점수,평균_리뷰수,편의시설_개수,숙소_설명_글자수,게스트_선호,숙소_등급
0,312761,https://www.airbnb.com/rooms/312761,"Very central location, in the middle of Shored...",1608226,within a few hours,100.0,t,t,t,Hackney,...,4.93,4.96,5.00,4.89,4.93,0.86,46,436,t,Top
11,38995,https://www.airbnb.com/rooms/38995,Welcome to my place! I offer a large room in a...,167281,within a few hours,100.0,t,t,t,Southwark,...,5.00,4.96,4.98,4.74,4.89,0.92,23,495,t,Top
16,183128,https://www.airbnb.com/rooms/183128,Spacious and comfortable double room with the ...,878083,within an hour,100.0,t,t,t,Southwark,...,4.98,4.99,4.99,4.85,4.96,1.37,55,409,t,Top
24,203163,https://www.airbnb.com/rooms/203163,Beautifully designed self contained room with ...,460272,within an hour,100.0,t,t,t,Southwark,...,4.92,4.94,4.96,4.89,4.84,0.77,46,493,t,Top
30,73431,https://www.airbnb.com/rooms/73431,The apartment is on the top floor of a portere...,383525,within an hour,100.0,t,t,t,Lambeth,...,4.97,4.91,4.90,4.75,4.79,0.71,33,155,t,Top
33,338354,https://www.airbnb.com/rooms/338354,"A light, comfortable room in a traditional Vic...",1719023,within an hour,100.0,t,t,t,Lewisham,...,4.93,4.98,4.96,4.75,4.91,0.79,24,235,t,Top
38,81635,https://www.airbnb.com/rooms/81635,"""Best views in London - Far exceeded expectati...",443041,within a few hours,100.0,t,t,t,Islington,...,4.89,4.95,4.94,4.89,4.89,0.50,24,226,t,Top
43,84532,https://www.airbnb.com/rooms/84532,our flat is overlooking the river Lea and it's...,462165,within a few hours,100.0,t,t,t,Hackney,...,4.97,4.93,4.99,4.70,4.96,1.29,42,311,t,Top
49,263822,https://www.airbnb.com/rooms/263822,This is a terraced ground floor flat that was ...,1384320,within an hour,100.0,f,t,t,Newham,...,3.98,4.73,4.80,4.24,4.29,0.30,25,412,f,Low
55,264781,https://www.airbnb.com/rooms/264781,"A two bedroom, 2 bathroom ground floor apartme...",1389063,within an hour,100.0,f,t,t,Lewisham,...,3.75,4.88,4.63,4.50,4.13,0.10,28,227,f,Low


# EDA_위치, 위도, 경도 시각화

## Inner London / Outer London

- 'Outer_London' = 'Sutton', 'Havering', 'Bexley', 'Barking and Dagenham', 'Harrow', 'Kingston upon Thames', 'Redbridge', 'Bromley', 'Enfield', 'Hillingdon', 'Merton', 'Hounslow', 'Richmond upon Thames', 'Croydon', 'Waltham Forest', 'Haringey', 'Barnet', 'Ealing', 'Newham', 'Brent'


- 'Inner_London' = 'Westminster', 'Tower Hamlets', 'Kensington and Chelsea', 'Camden', 'Hackney', 'Southwark', 'Lambeth', 'Islington', 'Wandsworth', 'Hammersmith and Fulham', 'Lewisham', 'Greenwich'


- 'City_of_London' = 'City of London'

In [30]:
import pandas as pd
import plotly.express as px


# 각 숙소 지역 정의
outer_london = ['Sutton', 'Havering', 'Bexley', 'Barking and Dagenham', 'Harrow', 
                'Kingston upon Thames', 'Redbridge', 'Bromley', 'Enfield', 'Hillingdon', 
                'Merton', 'Hounslow', 'Richmond upon Thames', 'Croydon', 'Waltham Forest', 
                'Haringey', 'Barnet', 'Ealing', 'Newham', 'Brent']
inner_london = ['Westminster', 'Tower Hamlets', 'Kensington and Chelsea', 'Camden', 
                'Hackney', 'Southwark', 'Lambeth', 'Islington', 'Wandsworth', 
                'Hammersmith and Fulham', 'Lewisham', 'Greenwich']
city_of_london = ['City of London']


# 지역 구분 함수 선언
def get_group(지역):
    if 지역 in outer_london:
        return 'Outer_London'
    elif 지역 in inner_london:
        return 'Inner_London'
    elif 지역 in city_of_london:
        return 'City_of_London'
    else:
        return 'Unknown'

# 시각화 함수 선언
def loca_viz(df, heights):
    # '숙소_지역' 컬럼의 각 유니크한 값별 데이터 개수 계산
    지역_데이터_개수 = df['숙소_지역'].value_counts().reset_index()
    지역_데이터_개수.columns = ['숙소_지역', '숙소_수']

    # 각 지역별 그룹 정의
    지역_데이터_개수['대분류'] = 지역_데이터_개수['숙소_지역'].apply(get_group)
    
    # 전체 데이터 개수
    전체_데이터_개수 = len(df)

    # 각 지역의 데이터 비율 계산
    지역_데이터_개수['비율(%)'] = (지역_데이터_개수['숙소_수'] / 전체_데이터_개수) * 100

    # Plotly를 이용한 가로 바 그래프 그리기
    fig = px.bar(
        지역_데이터_개수,
        x='숙소_수',
        y='숙소_지역',
        color='대분류',
        text='비율(%)',
        labels={'숙소_수': '숙소 수', '숙소_지역': '숙소 지역', '대분류': '대분류'},
        title='각 지역별 숙소 수 및 비율'
    )

    # 비율을 텍스트로 표시
    fig.update_traces(texttemplate='%{text:.2f}%', textposition='outside')

    # 그래프 레이아웃 업데이트
    fig.update_layout(
        xaxis_title='숙소 수',
        yaxis_title='숙소 지역',
        height=heights,
        yaxis=dict(categoryorder='total ascending', automargin=True),
        legend_title_text='대분류'
    )
    
    # 그래프 출력
    fig.show()


In [31]:
loca_viz(london_lists, 1200)

하위 지역은 런던중심에서 외곽 지역에 해당 (대부분 현지인 주거지역)
- Barking and Dagenham(바킹 앤드 다게넘): <span style="color:yellow">동쪽 외곽</span>에 위치 / <span style="color:yellow">공업 지대와 주거 지역이 혼재</span> 다문화 지역
- Havering(해버링): <span style="color:yellow">동쪽 외곽</span>에 위치 / <span style="color:yellow">주거 지역 느낌</span> 평온하고 넓은 녹지가 있다.
- Sutton(서턴): <span style="color:yellow">남쪽 외곽</span>에 위치 / <span style="color:yellow">주거 지역</span> (가족들이 살기 좋은 환경을 제공)
- City of London: <span style="color:yellow">북동쪽</span>에 위치 / <span style="color:yellow">런던의 금융 중심지</span> (고층 건물과 역사적인 랜드마크가 혼재)
- Bromley(브롬리): <span style="color:yellow">남동쪽 외곽</span>에 위치 / <span style="color:yellow"> 큰 주거 지역</span> (녹지가 많고 가족들이 많이 거주)
- Hillingdon(힐링던): <span style="color:yellow">서쪽 외곽</span>에 위치 / <span style="color:yellow">히드로 공항이 있다.</span>
- Redbridge(레드브리지): <span style="color:yellow">북동쪽</span>에 위치 / <span style="color:yellow">주거 지역</span> (다양한 커뮤니티와 녹지가 있다)
- Harrow(해로): <span style="color:yellow">북서쪽</span>에 위치 / <span style="color:yellow">주거 지역</span> (좋은 교육 시설이 많음)

<h4>숙소지역 특징 by 예진님</h4>

1. Hackney(해크니): <span style="color:yellow">동런던</span>에 위치한 트렌디한 지역 / <span style="color:yellow">예술가와 창의적</span>인 커뮤니티가 많이 거주

2. Islington(이슬링턴): <span style="color:yellow">중북부</span>에 위치 / <span style="color:yellow">고급 주거 지역</span> (고풍스러운 타운하우스와 다양한 음식점, 극장)

3. Kensington and Chelsea(켄싱턴 앤드 첼시): <span style="color:yellow">서쪽</span>에 위치 / <span style="color:yellow">가장 부유한 지역</span> (고급 부티크와 박물관, 아름다운 주택)

4. Lambeth(램버스): <span style="color:yellow">남쪽</span>위치한 지역 / 런던 아이와 사우스뱅크 센터 같은 <span style="color:yellow">주요 관광지</span> (다양하고 활기찬 커뮤니티)

5. Brent(브렌트): <span style="color:yellow">북서쪽</span>위치 / <span style="color:yellow">스포츠와 음악 이벤트</span> (웸블리 스타디움이 있는 곳)

6. Southwark(사우스워크): <span style="color:yellow">남동쪽</span> 위치 /   <span style="color:yellow">문화 관광 거리</span> (테이트 모던 미술관과 버러 마켓이 유명)

7. Westminster(웨스트민스터): <span style="color:yellow">중심부</span>위치 / <span style="color:yellow">정부 기관 지역</span> (국회의사당, 버킹엄 궁전, 트라팔가 광장)

8. Wandsworth(원스워스):<span style="color:yellow">남서쪽</span>위치 / 
<span style="color:yellow">주거 지역</span> (큰 공원과 가족 친화적인 분위기가 특징)

9. Tower Hamlets(타워햄리츠): <span style="color:yellow">동쪽</span> 위치 / 런던 타워와 카나리 워프 같은 <span style="color:yellow">역사적이고 상업적인 중심지</span>

10. Richmond upon Thames(리치먼드): <span style="color:yellow">남서쪽</span>에 위치 / <span style="color:yellow">풍요롭고 피크닉 장소</span> (리치몬드 공원과 템스 강이 유명)

11. Newham(뉴험): <span style="color:yellow">동쪽</span>에 위치 / <span style="color:yellow">다문화 지역</span> (올림픽 공원과 웨스트필드 쇼핑센터)

12. Camden(캠던): <span style="color:yellow">북부</span>에 위치 / <span style="color:yellow">보헤미안 지역</span> (캠든 마켓과 음악 공연장이 많음)

13. Haringey(하린지): <span style="color:yellow">북쪽</span>에 위치 / <span style="color:yellow">주거 지역</span> (알렉산드라 팰리스와 큰 공원)

14. Enfield(엔필드): <span style="color:yellow">북쪽 외곽</span>에 위치 / <span style="color:yellow">자연적인 지역</span> (넓은 녹지와 가족 친화적인 분위기가 특징)

15. Hammersmith and Fulham(해머스미스 앤드 풀럼): <span style="color:yellow">서쪽</span>에 위치 / <span style="color:yellow">국제 기업의 사무소 지역</span> (템스 강변의 아름다운 경치와 고급 주택들)

16. Waltham Forest(월섬 포레스트): <span style="color:yellow">북동쪽</span>에 위치 / <span style="color:yellow">녹지와 공원 지역</span> (예술과 문화적 활동이 활발한 곳)

17. Barnet(바넷): <span style="color:yellow">북서쪽</span>에 위치 / <span style="color:yellow">큰 주거 지역</span> (가족과 함께 살기 좋은 환경을 제공)

18. Hounslow(하운슬로): <span style="color:yellow">서쪽 외곽</span>에 위치 / <span style="color:yellow">히드로 공항과 가까운</span>특징 (교통이 편리)

19. Barking and Dagenham(바킹 앤드 다게넘): <span style="color:yellow">동쪽 외곽</span>에 위치 / <span style="color:yellow">공업 지대와 주거 지역이 혼재</span> 다문화 지역

20. Croydon(크로이던): <span style="color:yellow">남쪽</span>에 위치 / <span style="color:yellow">큰 상업 중심지</span> (쇼핑몰과 비즈니스 허브가 많음)

21. Lewisham(루이섬): <span style="color:yellow">남동쪽</span>에 위치 / <span style="color:yellow">다양한 문화적 배경을 가진 사람들이 거주</span> (활기찬 시장이 있다)

22. Ealing(일링): <span style="color:yellow">서쪽</span>에 위치 / <span style="color:yellow">가족 친화적인 분위기</span> (철도역이 있고 큰 공원)

23. Merton(머튼): <span style="color:yellow">남서쪽</span>에 위치 / <span style="color:yellow">윔블던 테니스 챔피언십이 열리는 윔블던이 있음</span> 

24. City of London: <span style="color:yellow">북동쪽</span>에 위치 / <span style="color:yellow">런던의 금융 중심지</span> (고층 건물과 역사적인 랜드마크가 혼재)

25. Greenwich(그리니치): <span style="color:yellow">남동쪽</span>에 위치 / <span style="color:yellow">그리니치 천문대 유명</span> (해양 박물관이 유명, 관광지)

26. Hillingdon(힐링던): <span style="color:yellow">서쪽 외곽</span>에 위치 / <span style="color:yellow">히드로 공항이 있다.</span>

27. Bromley(브롬리): <span style="color:yellow">남동쪽 외곽</span>에 위치 / <span style="color:yellow"> 큰 주거 지역</span> (녹지가 많고 가족들이 많이 거주)

28. Havering(해버링): <span style="color:yellow">동쪽 외곽</span>에 위치 / <span style="color:yellow">주거 지역 느낌</span> 평온하고 넓은 녹지가 있다.

29. Harrow(해로): <span style="color:yellow">북서쪽</span>에 위치 / <span style="color:yellow">주거 지역</span> (좋은 교육 시설이 많음)

30. Bexley(벡슬리): <span style="color:yellow">남동쪽 외곽</span>에 위치 / <span style="color:yellow">주거 지역</span> 가족 친화적인 분위기와 공원이 많습니다.

31. Redbridge(레드브리지): <span style="color:yellow">북동쪽</span>에 위치 / <span style="color:yellow">주거 지역</span> (다양한 커뮤니티와 녹지가 있다)

32. Kingston upon Thames(킹스턴): <span style="color:yellow">남서쪽</span>에 위치 / <span style="color:yellow">쇼핑 지역이 유명</span> (템스 강변의 아름다운 경치)

33. Sutton(서턴): <span style="color:yellow">남쪽 외곽</span>에 위치 / <span style="color:yellow">주거 지역</span> (가족들이 살기 좋은 환경을 제공)

In [32]:
# 지역 대분류별 숙소 수 확인 함수 선언
def loca_category(df):
    # '숙소_지역' 컬럼의 각 유니크한 값별 데이터 개수 계산
    지역_데이터_개수 = df['숙소_지역'].value_counts().reset_index()
    지역_데이터_개수.columns = ['숙소_지역', '숙소_수']

    # 각 지역별 그룹 정의
    지역_데이터_개수['대분류'] = 지역_데이터_개수['숙소_지역'].apply(get_group)
    
    # 전체 데이터 개수
    전체_데이터_개수 = len(df)

    # 각 지역의 데이터 비율 계산
    지역_데이터_개수['비율(%)'] = (지역_데이터_개수['숙소_수'] / 전체_데이터_개수) * 100
    
    # 각 그룹별 데이터 개수 계산
    그룹_데이터_개수 = 지역_데이터_개수.groupby('대분류')['숙소_수'].sum().reset_index()
    그룹_데이터_개수.columns = ['지역', '총_숙소_개수']
    그룹_데이터_개수['비율(%)'] = (그룹_데이터_개수['총_숙소_개수'] / 전체_데이터_개수) * 100

    # 결과 출력
    print(그룹_데이터_개수)

In [33]:
loca_category(london_lists)

               지역  총_숙소_개수      비율(%)
0  City_of_London      259   0.832771
1    Inner_London    22071  70.965564
2    Outer_London     8771  28.201666


In [34]:
loca_viz(london_toplow, 1200)

In [35]:
loca_category(london_toplow)

               지역  총_숙소_개수      비율(%)
0  City_of_London       52   0.734152
1    Inner_London     4851  68.487929
2    Outer_London     2180  30.777919


## 위도 / 경도

In [36]:
# !pip install folium

In [37]:
import folium
from folium.plugins import MarkerCluster

# 런던 중심부의 위도와 경도
london_latitude = 51.5074
london_longitude = -0.1278

# 런던 지도 생성
map_london = folium.Map(location=[london_latitude, london_longitude], zoom_start=12)

# 랜드마크 위치 데이터
# landmarks = {
#     "런던 아이": (51.5033, -0.1196),
#     "타워 브리지": (51.5055, -0.0754),
#     "버킹엄 궁전": (51.5014, -0.1419),
#     "대영 박물관": (51.5194, -0.1270),
#     "테이트 모던": (51.5076, -0.0994),
#     "세인트 폴 대성당": (51.5138, -0.0984),
#     "웨스트민스터 사원": (51.4993, -0.1273),
#     "피카딜리 서커스": (51.5101, -0.1342),
#     "하이드 파크": (51.5073, -0.1657),
#     "캠든 마켓": (51.5416, -0.1469)
# }

landmarks = {
    "런던 아이": (51.5033, -0.1196),
    "타워 브리지": (51.5055, -0.0754),
    "버킹엄 궁전": (51.5014, -0.1419),
    "대영 박물관": (51.5194, -0.1270),
    "테이트 모던": (51.5076, -0.0994),
    "세인트 폴 대성당": (51.5138, -0.0984),
    "웨스트민스터 사원": (51.4993, -0.1273),
    "피카딜리 서커스": (51.5101, -0.1342),
    "하이드 파크": (51.5073, -0.1657),
    "캠든 마켓": (51.5416, -0.1469),
    "빅벤": (51.5007, -0.1246),
    "내셔널 갤러리": (51.5089, -0.1283),
    "자연사 박물관": (51.4967, -0.1764),
    "타워 오브 런던": (51.5081, -0.0759),
    "코벤트 가든": (51.5115, -0.1234),
    "리젠트 파크": (51.5313, -0.1569),
    "셔드": (51.5045, -0.0865),
    "킹스 크로스 세인트 판크라스": (51.5308, -0.1238)
}



# 각 랜드마크에 대한 마커 추가
for landmark, (lat, lng) in landmarks.items():
    folium.Marker(
        location=[lat, lng],
        icon=folium.Icon(icon='star', color='blue'),
        popup=landmark
    ).add_to(map_london)

# 클러스터 객체 생성
marker_cluster = MarkerCluster().add_to(map_london)

# df_guest_prefer의 위치 표시 (클러스터에 추가)
for idx, row in london_toplow[(london_toplow['숙소_등급'] == 'Top')].iterrows():
    folium.Marker(
        location=[row['위도'], row['경도']],
        icon=folium.Icon(color='green', icon='info-sign'),
        popup='Guest Prefer'
    ).add_to(marker_cluster)

# df_non_guest_prefer의 위치 표시 (클러스터에 추가)
for idx, row in london_toplow[(london_toplow['숙소_등급'] == 'Low')].iterrows():
    folium.Marker(
        location=[row['위도'], row['경도']],
        icon=folium.Icon(color='red', icon='info-sign'),
        popup='Non-Guest Prefer'
    ).add_to(marker_cluster)

# 지도를 HTML 파일로 저장
map_london.save('london_landmarks_map.html')

In [38]:
# !pip install geopy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 125.4/125.4 kB 3.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.3/40.3 kB 4.1 MB/s eta 0:00:00


In [39]:
from geopy.distance import geodesic

# 특정 랜드마크와의 거리 계산 함수 정의
def count_accommodations_within_distance(df, landmark_coords, max_distance_km):
    count = 0
    for idx, row in df.iterrows():
        # 숙소 위치
        accommodation_location = (row['위도'], row['경도'])
        # 랜드마크와 숙소 간의 거리 계산
        distance = geodesic(landmark_coords, accommodation_location).km
        # 특정 거리 이내에 있으면 카운트
        if distance <= max_distance_km:
            count += 1
    return count

# 각 랜드마크와의 거리 계산
landmark_distances = {}
for landmark, coords in landmarks.items():
    count_prefer = count_accommodations_within_distance(london_toplow[(london_toplow['숙소_등급'] == 'Top')], coords, 1)  # 1km 이내
    count_non_prefer = count_accommodations_within_distance(london_toplow[(london_toplow['숙소_등급'] == 'Low')], coords, 1)  # 1km 이내
    landmark_distances[landmark] = {'Guest Prefer': count_prefer, 'Non-Guest Prefer': count_non_prefer}

# 결과 출력
for landmark, counts in landmark_distances.items():
    print(f"{landmark}:  1km 내 게스트 선호: {counts['Guest Prefer']}, 게스트 불호: {counts['Non-Guest Prefer']}")


런던 아이:  1km 내 게스트 선호: 31, 게스트 불호: 33
타워 브리지:  1km 내 게스트 선호: 33, 게스트 불호: 33
버킹엄 궁전:  1km 내 게스트 선호: 11, 게스트 불호: 27
대영 박물관:  1km 내 게스트 선호: 102, 게스트 불호: 123
테이트 모던:  1km 내 게스트 선호: 35, 게스트 불호: 28
세인트 폴 대성당:  1km 내 게스트 선호: 16, 게스트 불호: 26
웨스트민스터 사원:  1km 내 게스트 선호: 20, 게스트 불호: 17
피카딜리 서커스:  1km 내 게스트 선호: 95, 게스트 불호: 102
하이드 파크:  1km 내 게스트 선호: 15, 게스트 불호: 50
캠든 마켓:  1km 내 게스트 선호: 43, 게스트 불호: 48
빅벤:  1km 내 게스트 선호: 21, 게스트 불호: 21
내셔널 갤러리:  1km 내 게스트 선호: 78, 게스트 불호: 83
자연사 박물관:  1km 내 게스트 선호: 70, 게스트 불호: 101
타워 오브 런던:  1km 내 게스트 선호: 34, 게스트 불호: 43
코벤트 가든:  1km 내 게스트 선호: 78, 게스트 불호: 86
리젠트 파크:  1km 내 게스트 선호: 19, 게스트 불호: 20
셔드:  1km 내 게스트 선호: 36, 게스트 불호: 44
킹스 크로스 세인트 판크라스:  1km 내 게스트 선호: 46, 게스트 불호: 91


In [46]:
import plotly.graph_objects as go
import pandas as pd
import numpy as np

# 랜드마크 거리 데이터 프레임 생성
landmark_df = pd.DataFrame(landmark_distances).T  # 행과 열 전환

# 위치 인덱스 생성
index = np.arange(len(landmark_df))

# Plotly 그래프 생성
fig = go.Figure()

# 각 그룹 별 막대 그래프 추가
fig.add_trace(go.Bar(
    x=landmark_df.index,
    y=landmark_df['Guest Prefer'],
    name='Guest Prefer',
    marker_color='green'
))

fig.add_trace(go.Bar(
    x=landmark_df.index,
    y=landmark_df['Non-Guest Prefer'],
    name='Non-Guest Prefer',
    marker_color='red'
))

# 레이아웃 설정
fig.update_layout(
    title='각 랜드마크 별 1km 이내 숙소 수',
    xaxis=dict(
        title='랜드마크',
        tickmode='array',
        tickvals=index,
        ticktext=landmark_df.index
    ),
    yaxis=dict(
        title='숙박 수'
    ),
    barmode='group',
    bargap=0.15,
    bargroupgap=0.1,
    height=600,
    legend=dict(
        x=0.8,
        y=1.0,
        bgcolor='rgba(255, 255, 255, 0)',
        bordercolor='rgba(255, 255, 255, 0)'
    ),
)

# 그래프 표시
fig.show()
